In [2]:
from skimage import io
import random
import os
import albumentations as A
import cv2
import matplotlib.pyplot as plt

##### Usefull paths

In [3]:
imgs_path = os.path.join('data', 'originals','imgs')
masks_path = os.path.join('data', 'originals', 'masks')
augmented_imgs_path = os.path.join('data','augmented', 'aug_imgs')
augmented_masks_path = os.path.join('data', 'augmented', 'aug_masks')

##### Resize images

In [11]:
# Tamaño deseado para las imágenes
desired_width = 256
desired_height = 256

# Iterar sobre todas las imágenes en el directorio de entrada
for img in os.listdir(imgs_path):
    if img.endswith('.png') or img.endswith('.jpeg'):
        # Leer la imagen
        image = io.imread(os.path.join(imgs_path, img))
        resized_img = cv2.resize(image, (desired_width, desired_height))
        cv2.imwrite(os.path.join(imgs_path, img), resized_img)

# Iterar sobre todas las máscaras en el directorio de entrada
for mask in os.listdir(masks_path):
    if mask.endswith('.png') or mask.endswith('.jpeg'):
        # Leer la máscara
        mask_img = io.imread(os.path.join(masks_path, mask))
        resized_mask = cv2.resize(mask_img, (desired_width, desired_height))
        cv2.imwrite(os.path.join(masks_path, mask), resized_mask)
    

##### Create and save augmented images

In [12]:
images_to_generate = 30

aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate90(p=0.5),
    A.Rotate180(p=0.5),
    A.Rotate270(p=0.5),
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2)
])

images = []
masks = []


i = 1
num_images = len(os.listdir(imgs_path))
while i <= images_to_generate:
    number = random.randint(0, num_images - 1)
    try:
        image = os.path.join(imgs_path, f'{number}_RGB.png')
        mask = os.path.join(masks_path, f'{number}_mask.png')
        original_image = io.imread(image)
        original_mask = io.imread(mask)
        
        augmented = aug(image=original_image, mask=original_mask)
        transformed_image = augmented['image']
        transformed_mask = augmented['mask']

        new_image_path= "%s/%s.png" %(augmented_imgs_path, i)
        new_mask_path = "%s/%s.png" %(augmented_masks_path, i)
        io.imsave(new_image_path, transformed_image)
        io.imsave(new_mask_path, transformed_mask)
    except FileNotFoundError:
        continue
    i =i+1

**This code has been obtained from [this video](https://www.youtube.com/watch?v=J_XSd_u_Yew&t=359s)